# PureActionState

## Example: Create GENERATE state using PureActionState

```lua
stateDiagram-v2
    INIT --> GENERATE
    GENERATE --> FINAL
```

```mermaid
stateDiagram-v2
    Direction LR
    INIT --> GENERATE
    GENERATE --> FINAL
```


In [1]:
from gai.asm import AsyncStateMachine

async def generate_action(state):
    
    from openai import AsyncOpenAI
    client = AsyncOpenAI()
    
    # Import data from state_bag
    agent_name = state.machine.state_bag.get("name", "Assistant")
    user_message = state.machine.state_bag.get("user_message", "If you are seeing this, that means I have forgotten to add a user message. Remind me.")
    
    from openai import AsyncOpenAI
    client = AsyncOpenAI()
    
    # Execute
    
    state.machine.monologue.add_user_message(content=f"Your name is {agent_name}. You are a helpful assistant.{agent_name}, {user_message}")
    response = await client.chat.completions.create(
        model="gpt-4.1",
        messages=state.machine.monologue.list_chat_messages(),
        max_tokens=50,
        stream=True,
    )
    
    async def streamer():
        content = ""
        async for chunk in response:
            chunk = chunk.choices[0].delta.content
            if isinstance(chunk,str) and chunk:
                content += chunk
                yield chunk
        state.machine.monologue.add_assistant_message(content=content)

    state.machine.state_bag["streamer"] = streamer()

## Step 1: INIT

with AsyncStateMachine.StateMachineBuilder("""
    INIT --> GENERATE
    GENERATE --> FINAL
    """
    ) as builder:
    fsm = builder.build({
        "INIT": {
            "input_data": {
                "user_message": "Hello, world!",
                "dialogue_messages": []
            },
        },
        "GENERATE": {
            "module_path": "gai.asm.states",
            "class_name": "PureActionState",
            "title": "GENERATE",
            "action": "generate",
            "output_data": ["streamer"]            
        },
        "FINAL": {
            "output_data": ["monologue"],
        }
    },generate=generate_action)

## Step 2: INIT --> GENERATE

await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    print(chunk, end='', flush=True)
print("\n\n")

## Step 3: GENERATE --> FINAL
await fsm.run_async()

## Step 4: Print the state history
print("State History:")
for state in fsm.state_history:
    print(f"State: {state['state']}")
    print(f"- input: {state['input']}")
    print(f"- output: {state['output']}")
    print("-" * 20)

Hello, world! 🌎 How can I assist you today?


State History:
State: INIT
- input: {'user_message': 'Hello, world!', 'dialogue_messages': [], 'monologue': <gai.asm.monologue.Monologue object at 0x7292ca1186d0>, 'step': 0, 'time': datetime.datetime(2025, 6, 14, 3, 54, 21, 209359), 'name': 'Assistant'}
- output: {'name': 'Assistant', 'monologue': <gai.asm.monologue.Monologue object at 0x7292cae33d90>, 'step': 0, 'time': datetime.datetime(2025, 6, 14, 3, 54, 21, 209419)}
--------------------
State: GENERATE
- input: {'name': 'Assistant', 'monologue': <gai.asm.monologue.Monologue object at 0x7292ca1190f0>, 'step': 1, 'time': datetime.datetime(2025, 6, 14, 3, 54, 21, 209602)}
- output: {'streamer': <async_generator object generate_action.<locals>.streamer at 0x7292c94d9240>, 'name': 'Assistant', 'monologue': <gai.asm.monologue.Monologue object at 0x7292c94b8bb0>, 'step': 1, 'time': datetime.datetime(2025, 6, 14, 3, 54, 22, 300207)}
--------------------
State: FINAL
- input: {'streamer': <asy